In [1]:
!pip install openai

In [6]:
!pip install openai langchain langchain-openai nbformat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [9]:
!pip uninstall -y openai
!pip install --upgrade openai

Found existing installation: openai 1.57.2
Uninstalling openai-1.57.2:
  Successfully uninstalled openai-1.57.2
  Using cached openai-1.57.2-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.57.2-py3-none-any.whl (389 kB)


In [10]:
from google.colab import drive
import os
import json
from typing import Optional

drive.mount('/content/drive')
config_dir = '/content/drive/MyDrive/colab_config'
key_path = f'{config_dir}/openai_key.json'

def save_api_key(key: str) -> None:
    """Save OpenAI API key to Drive."""
    os.makedirs(config_dir, exist_ok=True)
    with open(key_path, 'w') as f:
        json.dump({'api_key': key}, f)

def load_api_key() -> Optional[str]:
    """Load OpenAI API key from Drive."""
    try:
        with open(key_path, 'r') as f:
            data = json.load(f)
            return data['api_key']
    except:
        return None

def test_openai_connection(api_key: str) -> bool:
    """Test if the OpenAI API key works."""
    try:
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": "Test message"}],
            max_tokens=5
        )
        return True
    except Exception as e:
        print(f"OpenAI API test failed: {str(e)}")
        return False

def setup_openai_key() -> bool:
    """Setup OpenAI API key and verify it works."""
    api_key = load_api_key()

    if not api_key:
        print("Please enter your OpenAI API key:")
        new_key = input().strip()  # Added strip() to remove any accidental whitespace
        api_key = new_key

    if test_openai_connection(api_key):
        save_api_key(api_key)
        os.environ["OPENAI_API_KEY"] = api_key
        print("OpenAI API key verified successfully!")
        return True
    return False

if 'OPENAI_API_KEY' in os.environ:
    del os.environ['OPENAI_API_KEY']

if not setup_openai_key():
    print("Would you like to try with a new API key? (yes/no)")
    response = input()
    if response.lower() == 'yes':
        if os.path.exists(key_path):
            os.remove(key_path)
        if not setup_openai_key():
            print("Failed to setup OpenAI API key. Please check your key and try again.")
else:
    print("Setup completed successfully! You can now use OpenAI in your notebook.")
try:
    from openai import OpenAI
    client = OpenAI()  # This will use the environment variable
    print("\nTesting connection with a simple query...")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Say hello"}],
        max_tokens=10
    )
    print("Test successful! Response:", response.choices[0].message.content)
except Exception as e:
    print(f"Error during final verification: {str(e)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
OpenAI API key verified successfully!
Setup completed successfully! You can now use OpenAI in your notebook.

Testing connection with a simple query...
Test successful! Response: Hello! How can I assist you today?


### Part A: Build a code understanding model.

In [ ]:
import json
import nbformat
from openai import OpenAI
import os
from pathlib import Path
from typing import List, Dict, Optional

class JupyterNotebookAnalyzer:
    def __init__(self, api_key: Optional[str] = None):
        if api_key:
            os.environ["OPENAI_API_KEY"] = api_key
        self.client = OpenAI()
        self.notebook_content = None

    def load_notebook(self, notebook_path: str) -> dict:
        try:
            with open(notebook_path, 'r', encoding='utf-8') as f:
                self.notebook_content = nbformat.read(f, as_version=4)
            return self.extract_notebook_content()
        except Exception as e:
            raise ValueError(f"Error loading notebook: {str(e)}")

    def extract_notebook_content(self) -> dict:
        content = {
            'code_cells': [],
            'markdown_cells': [],
            'imports': [],
            'functions': [],
            'model_architecture': [],
            'training_code': [],
        }
        for cell in self.notebook_content.cells:
            if cell.cell_type == 'code':
                code = cell.source
                content['code_cells'].append(code)
                if 'import' in code:
                    content['imports'].append(code)
                elif 'def ' in code:
                    content['functions'].append(code)
                elif any(term in code.lower() for term in ['model', 'sequential', 'conv2d', 'dense']):
                    content['model_architecture'].append(code)
                elif any(term in code.lower() for term in ['fit', 'compile', 'train', 'optimizer']):
                    content['training_code'].append(code)
            elif cell.cell_type == 'markdown':
                content['markdown_cells'].append(cell.source)
        return content

    def analyze_notebook(self, question: str) -> str:
        if not self.notebook_content:
            raise ValueError("No notebook has been loaded yet")
        context = "\n=== IMPORTS ===\n"
        context += "\n".join(self.extract_notebook_content()['imports'])
        context += "\n\n=== MODEL ARCHITECTURE ===\n"
        context += "\n".join(self.extract_notebook_content()['model_architecture'])
        context += "\n\n=== TRAINING CODE ===\n"
        context += "\n".join(self.extract_notebook_content()['training_code'])
        context += "\n\n=== OTHER CODE ===\n"
        context += "\n".join([cell for cell in self.extract_notebook_content()['code_cells']
                            if cell not in self.extract_notebook_content()['imports'] and
                               cell not in self.extract_notebook_content()['model_architecture'] and
                               cell not in self.extract_notebook_content()['training_code']])

        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages=[
                    {"role": "system", "content": """You are an expert in deep learning and PyTorch/TensorFlow analysis.
                    Focus on:
                    - Model architecture and design choices
                    - Training methodology and hyperparameters
                    - Data preprocessing and augmentation
                    - Performance optimization opportunities
                    - Best practices in deep learning"""},
                    {"role": "user", "content": f"""
                    Please analyze this Fashion MNIST notebook and answer the question:

                    {context}

                    Question: {question}
                    """}
                ],
                temperature=0.2,
                max_tokens=2000
            )
            return response.choices[0].message.content

        except Exception as e:
            return f"Error analyzing notebook: {str(e)}"

    def get_model_summary(self) -> str:
        return self.analyze_notebook("""
        Provide a detailed summary of the model architecture, including:
        - Layer structure and parameters
        - Design choices and their implications
        - Potential improvements
        """)

    def get_training_analysis(self) -> str:
        return self.analyze_notebook("""
        Analyze the training setup, including:
        - Optimizer choice and hyperparameters
        - Loss function
        - Batch size and epochs
        - Learning rate strategy
        - Data preprocessing steps
        """)

### Upload your own custom code files to the model and ask questions based on the code file as context.

In [2]:
def main():
    analyzer = JupyterNotebookAnalyzer()
    notebook_path = '/content/drive/My Drive/Deep Learning/Assignment/HW12/FMNIST.ipynb'
    try:
        # Load the notebook
        analyzer.load_notebook(notebook_path)
        # Get model architecture summary
        print("\nModel Architecture Summary:")
        print(analyzer.get_model_summary())
        # Get training analysis
        print("\nTraining Setup Analysis:")
        print(analyzer.get_training_analysis())
        # Ask specific questions
        questions = [
            "What is the overall approach used for the Fashion MNIST classification?",
            "How could the model architecture be improved?",
            "Are there any potential overfitting issues?",
            "What data augmentation techniques are used?",
            "How does the learning rate schedule work?"
        ]
        for question in questions:
            print(f"\nQuestion: {question}")
            response = analyzer.analyze_notebook(question)
            print(f"Response: {response}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
if __name__ == "__main__":
    main()


Model Architecture Summary:
The provided notebook outlines the development and evaluation of a neural network model for the Fashion MNIST dataset using PyTorch. Below is a detailed analysis of the model architecture, design choices, and potential areas for improvement.

### Model Architecture

The model architecture is defined in the `MLP` class, which represents a simple Multi-Layer Perceptron (MLP) with the following layers:

1. **Input Layer**: The input layer is a fully connected (linear) layer with 784 inputs (28x28 images flattened) and 256 outputs. This is an increase from an earlier version of the model which had 128 outputs. Increasing the number of outputs (neurons) in this layer allows the model to potentially learn more complex representations of the input data.

2. **Hidden Layer 1**: This is another fully connected layer with 256 inputs and 64 outputs. The increase in the number of inputs (from 128 to 256) matches the increased output size of the previous layer. This lay

In [3]:
!pip install langchain langchain_community langchain_openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.21
    Uninstalling langchain-core-0.3.21:
      Successfully uninstalled langchain-core-0.3.21
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.9
    Uninstalling langchain-0.3.9:
      Successfully uninstalled langchain-0.3.9


### Part B: Write a chatbot prompt to iteratively create a sequence of chats on one particular custom data.



In [ ]:
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from google.colab import drive
import json
from typing import List, Dict
import os
from tenacity import retry, stop_after_attempt, wait_exponential

class EfficientSequentialChatbot:
    def __init__(self):
        try:
            self.llm = ChatOpenAI(
                temperature=0.5,
                model_name="gpt-3.5-turbo"
            )
            self.memory = ConversationBufferMemory(
                memory_key="chat_history",
                return_messages=True
            )
            self.conversation_history = []
        except Exception as e:
            print(f"Initialization error: {str(e)}")
            raise

    def load_drive_document(self, file_path: str):
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                print(f"Document loaded successfully")
            text_splitter = CharacterTextSplitter(
                chunk_size=300,
                chunk_overlap=50
            )
            chunks = text_splitter.split_text(text)
            embeddings = OpenAIEmbeddings()
            self.vectorstore = FAISS.from_texts(chunks, embeddings)
            self.qa_chain = ConversationalRetrievalChain.from_llm(
                self.llm,
                self.vectorstore.as_retriever(search_kwargs={"k": 1}),  # Limit to 1 chunk for efficiency
                memory=self.memory
            )
            return True
        except Exception as e:
            print(f"Error loading document: {str(e)}")
            return False

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
    def get_response(self, question: str) -> str:
        try:
            response = self.qa_chain({"question": question})
            return response["answer"]
        except Exception as e:
            print(f"Error getting response: {str(e)}")
            return "I apologize, but I encountered an error. Please try asking your question again."

    def chat_sequence(self):
        print("\nStarting interactive chat sequence. Type 'exit' to end, 'summary' for current summary.")
        question_count = 0
        while True:
            try:
                user_input = input("\nYour question (or 'exit'/'summary'): ").strip()
                if user_input.lower() == 'exit':
                    break
                elif user_input.lower() == 'summary':
                    print("\nCurrent Conversation Summary:")
                    print(self.get_summary())
                    continue
                question_count += 1
                print(f"\nQuestion {question_count}: {user_input}")
                response = self.get_response(user_input)
                print(f"Answer: {response}")
                self.conversation_history.append({
                    "question_number": question_count,
                    "question": user_input,
                    "answer": response
                })
            except Exception as e:
                print(f"Error during chat: {str(e)}")
                print("You can continue with your next question.")

    def get_summary(self) -> str:
        if not self.conversation_history:
            return "No conversation to summarize yet."
        conversation = "\n".join([
            f"Q: {entry['question']}\nA: {entry['answer']}"
            for entry in self.conversation_history
        ])
        summary_prompt = """Analyze this conversation and provide a structured summary that includes:
        1. Main Topics: What were the key subjects discussed?
        2. Progression: How did the understanding develop through the conversation?
        3. Key Insights: What were the important findings or revelations?
        4. Conclusions: What are the main takeaways?

        Please provide this analysis based on the following conversation:

        {conversation}
        """
        try:
            response = self.llm.invoke(summary_prompt)
            return response.content
        except Exception as e:
            print(f"Error generating analytical summary: {str(e)}")
            basic_summary = "\nBasic Conversation Summary:\n\n"
            basic_summary += f"Total questions asked: {len(self.conversation_history)}\n"
            basic_summary += "\nKey Questions Discussed:\n"
            for entry in self.conversation_history:
                basic_summary += f"- {entry['question']}\n"
            return basic_summary

    def save_conversation(self, filename: str = "chat_history.json"):
        try:
            output_path = os.path.join('/content/drive/My Drive/Deep Learning/Assignment/HW12', filename)
            data = {
                "conversation_history": self.conversation_history,
                "summary": self.get_summary()
            }
            with open(output_path, 'w') as f:
                json.dump(data, f, indent=2)
            print(f"\nConversation saved to: {output_path}")
        except Exception as e:
            print(f"Error saving conversation: {str(e)}")
            print("Try saving to a different location or checking file permissions.")

In [9]:
def main():
    try:
        drive.mount('/content/drive')
        # Initialize chatbot
        chatbot = EfficientSequentialChatbot()
        # Load document
        file_path = '/content/drive/My Drive/Deep Learning/Assignment/HW12/climate_change.txt'
        if not chatbot.load_drive_document(file_path):
            return
        # Start interactive chat
        print("\nChat with the document. Type 'exit' to end, 'summary' to see conversation summary.")
        chatbot.chat_sequence()
        # Final summary and save
        print("\nFinal Conversation Summary:")
        print(chatbot.get_summary())
        chatbot.save_conversation()
    except Exception as e:
        print(f"Main execution error: {str(e)}")
if __name__ == "__main__":
    main()

# What is the main topic of the document?
# How does this topic affect global systems?
# What solutions are proposed?
# How effective are these solutions?
# What are the future implications?

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Document loaded successfully

Chat with the document. Type 'exit' to end, 'summary' to see conversation summary.

Starting interactive chat sequence. Type 'exit' to end, 'summary' for current summary.

Your question (or 'exit'/'summary'): What is the main topic of the document?

Question 1: What is the main topic of the document?
Answer: The main topic of the document is climate change, specifically focusing on how it is a global challenge primarily caused by human activities increasing greenhouse gas concentrations in the atmosphere.

Your question (or 'exit'/'summary'): How does this topic affect global systems?

Question 2: How does this topic affect global systems?
Answer: Climate change, caused by human activities increasing greenhouse gas concentrations in the atmosphere, affects global systems in various ways. Some of the impacts include:

1. **Rising 